# 1. 기상청 API 데이터 수집

In [3]:
import requests
import pandas as pd
from io import StringIO
from datetime import datetime, timedelta
import xmltodict

# 날짜 리스트 생성 (2023년 1월 1일 ~ 24년 12월 31일)
start_date = datetime(2023, 1, 1)  # 2023년부터 시작
end_date = datetime(2024, 12, 31)
date_list = [(start_date + timedelta(days=i)).strftime('%Y%m%d') for i in range((end_date - start_date).days + 1)]

weather_records = []

col_names = [f'col{i}' for i in range(56)]  # 데이터 열 개수에 맞춤
url_weather = "https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php"

for date in date_list:
    params_weather = {
        "tm1": date,
        "tm2": date,
        "stn": "108",
        "help": "1",
        "authKey": "n7wd-Im8T368HfiJvO9-5Q"
    }
    try:
        response_weather = requests.get(url_weather, params=params_weather, timeout=10)
        response_weather.encoding = 'euc-kr'
        lines = response_weather.text.split('\n')
        lines = [line for line in lines if line and not line.startswith('#')]
        if not lines:
            continue
        csv_data = '\n'.join(lines)
        df_w = pd.read_csv(StringIO(csv_data), sep='\s+', header=None, names=col_names, engine='python')
        df_w = df_w.reset_index(drop=True)
        df_w['date'] = df_w['col0'].astype(str).str[:8]
        df_w['date_dt'] = pd.to_datetime(df_w['date'], format='%Y%m%d', errors='coerce')
        # 주요 변수만 추출
        df_w = df_w[['date_dt', 'col16', 'col21', 'col2']]
        df_w.columns = ['date_dt', 'TA_AVG', 'HM_AVG', 'WS_AVG']
        weather_records.append(df_w)
    except Exception as e:
        continue

if weather_records:
    df_weather_all = pd.concat(weather_records, ignore_index=True)
else:
    df_weather_all = pd.DataFrame()

print('기상청 데이터 수집 완료, 샘플 데이터:')
print(df_weather_all.head())
print('총 수집된 날짜 수:', df_weather_all['date_dt'].nunique())


기상청 데이터 수집 완료, 샘플 데이터:
     date_dt  TA_AVG  HM_AVG  WS_AVG
0 2023-01-01    -1.4     3.3     2.7
1 2023-01-02    -4.2     2.0     2.5
2 2023-01-03    -4.8     2.0     1.8
3 2023-01-04    -2.9     2.7     1.9
4 2023-01-05    -3.3     3.1     1.6
총 수집된 날짜 수: 731


# 2. 산불위험예보 목록정보 전처리

In [4]:
import pandas as pd

# 1. 기상청 데이터 준비 (이미 df_weather_all에 날짜별로 TA_AVG, HM_AVG, WS_AVG 있음)
# df_weather_all: ['date_dt', 'TA_AVG', 'HM_AVG', 'WS_AVG']

# 2. 산불위험예보 목록정보 불러오기 및 전처리
df_risk = pd.read_csv('산림청 국립산림과학원_대형산불위험예보목록정보_20250430.csv', encoding='euc-kr')
df_risk = df_risk.rename(columns={
    '예보일시': 'date',
    '시도명': 'province',
    '시군구명': 'city',
    '실효습도': 'effective_humidity',
    '풍속': 'wind_speed',
    '등급': 'risk_grade'
})
df_risk['date_dt'] = pd.to_datetime(df_risk['date'], errors='coerce').dt.date
df_risk['effective_humidity'] = pd.to_numeric(df_risk['effective_humidity'], errors='coerce')
df_risk['wind_speed'] = pd.to_numeric(df_risk['wind_speed'], errors='coerce')

# 3. 1차 병합 (날짜 기준)

In [5]:
df_weather_all['date_dt'] = pd.to_datetime(df_weather_all['date_dt']).dt.date
risk_weather = pd.merge(df_risk, df_weather_all, on='date_dt', how='inner')

print('1차 병합(기상+위험예보) 샘플:', risk_weather.head())

1차 병합(기상+위험예보) 샘플:                date province city 읍면동명  effective_humidity  wind_speed  \
0  2023-01-02 00:00       강원   양양  손양면                30.4         7.1   
1  2023-01-02 03:00       부산  금정구  노포동                32.5         8.4   
2  2023-01-02 03:00       전남   여수   국동                34.4         8.1   
3  2023-01-02 03:00       전남   여수  만흥동                34.4         8.8   
4  2023-01-02 03:00       전남   여수  봉계동                34.4         8.0   

  risk_grade     date_dt  TA_AVG  HM_AVG  WS_AVG  
0        주의보  2023-01-02    -4.2     2.0     2.5  
1        주의보  2023-01-02    -4.2     2.0     2.5  
2        주의보  2023-01-02    -4.2     2.0     2.5  
3        주의보  2023-01-02    -4.2     2.0     2.5  
4        주의보  2023-01-02    -4.2     2.0     2.5  


# 4. 산불상황관제시스템 통계데이터 전처리

In [6]:
# 4. 산불상황관제시스템 통계데이터 불러오기 및 전처리
df_fire = pd.read_csv('sanbul.csv', encoding='cp949')
df_fire = df_fire.rename(columns={
    '발생일시_년': 'year',
    '발생일시_월': 'month',
    '발생일시_일': 'day',
    '발생장소_시도': 'province',
    '발생장소_시군구': 'city'
})
df_fire['date_dt'] = pd.to_datetime(
    df_fire['year'].astype(str) + '-' +
    df_fire['month'].astype(str).str.zfill(2) + '-' +
    df_fire['day'].astype(str).str.zfill(2),
    errors='coerce'
).dt.date

# 일별, 지역별 산불 발생 건수 및 유무 집계
fire_group = df_fire.groupby(['date_dt', 'province', 'city']).size().reset_index(name='fire_count')
fire_group['fire_occurred'] = (fire_group['fire_count'] > 0).astype(int)

print('산불 발생 통계 샘플:', fire_group.head())


산불 발생 통계 샘플:       date_dt province city  fire_count  fire_occurred
0  2015-01-03       부산  해운대           1              1
1  2015-01-04       경북   안동           1              1
2  2015-01-04       전남   순천           1              1
3  2015-01-05       강원   강릉           1              1
4  2015-01-10       강원   강릉           1              1


# 5. 2차 병합 (날짜+지역 기준)

In [7]:
final_df = pd.merge(
    risk_weather,
    fire_group,
    on=['date_dt', 'province', 'city'],
    how='left'
)

# # 결측치 처리: 산불이 발생하지 않은 경우 0으로 채움

In [8]:
final_df['fire_occurred'] = final_df['fire_occurred'].fillna(0).astype(int)
final_df['fire_count'] = final_df['fire_count'].fillna(0).astype(int)

# feature 결측치 제거

In [9]:
features = ['TA_AVG', 'HM_AVG', 'WS_AVG', 'effective_humidity', 'wind_speed']
final_df = final_df.dropna(subset=features)

# 6. 타겟 변수 분포 확인

In [10]:
print('fire_occurred NaN 개수:', final_df['fire_occurred'].isna().sum())
print('타겟 분포:', final_df['fire_occurred'].value_counts())

fire_occurred NaN 개수: 0
타겟 분포: fire_occurred
0    6008
1     240
Name: count, dtype: int64


# 7. 데이터 불균형 처리 (0과 1이 모두 있을 때만 실행)

In [11]:
from imblearn.over_sampling import SMOTE

features = ['TA_AVG', 'HM_AVG', 'WS_AVG', 'effective_humidity', 'wind_speed']
X = final_df[features].astype(float)
y = final_df['fire_occurred']

# SMOTE 적용 (기존 resample 방식 대체)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [12]:
features = ['TA_AVG', 'HM_AVG', 'WS_AVG', 'effective_humidity', 'wind_speed']
X = final_df[features].astype(float)
y = final_df['fire_occurred']

In [13]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 8. 머신러닝 모델 학습

In [14]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled,
    test_size=0.2,
    random_state=42,
    stratify=y_resampled
)

# 모델 학습
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)

# 예측 및 평가
preds = model.predict(X_test)
print("정확도:", accuracy_score(y_test, preds))
print("\n분류 리포트:")
print(classification_report(y_test, preds))
print("\n혼동 행렬:")
print(confusion_matrix(y_test, preds))

정확도: 0.9825291181364393

분류 리포트:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1202
           1       0.97      0.99      0.98      1202

    accuracy                           0.98      2404
   macro avg       0.98      0.98      0.98      2404
weighted avg       0.98      0.98      0.98      2404


혼동 행렬:
[[1169   33]
 [   9 1193]]


In [15]:
result_df = X_test.copy()
result_df['actual_fire_occurred'] = y_test
result_df['predicted_fire_occurred'] = preds
print(result_df.head())

       TA_AVG  HM_AVG  WS_AVG  effective_humidity  wind_speed  \
11264    11.0    11.1     3.6           25.883018   12.369823   
3924      8.8     6.0    -9.0           41.200000    8.900000   
2587     11.0    11.1     3.6           39.600000    8.200000   
11593    12.9     4.2     1.8           38.400000    8.227590   
236      -6.6     1.0     4.6           30.800000    8.400000   

       actual_fire_occurred  predicted_fire_occurred  
11264                     1                        1  
3924                      0                        0  
2587                      1                        1  
11593                     1                        1  
236                       0                        0  


In [16]:
import joblib

# 이미 학습된 모델이 model 변수에 저장되어 있다고 가정
# 원하는 파일명으로 모델 저장 (예: 'xgb_fire_model_smote.pkl')
joblib.dump(model, 'xgb_fire_model_smote.pkl')

print("모델이 'xgb_fire_model_smote.pkl' 파일로 저장되었습니다.")


모델이 'xgb_fire_model_smote.pkl' 파일로 저장되었습니다.
